<a href="https://colab.research.google.com/github/sunny0103/DeepLearning_nlp_projects/blob/main/AIconnect_MRC/aiconnect_mrc_hyperparametersearch_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch] datasets evaluate optuna

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00


In [ ]:
!pip install -U datasets

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/e2/cf/db41e572d7ed958e8679018f8190438ef700aeb501b62da9e1eed9e4d69a/datasets-2.15.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Obtaining dependency information for huggingface-hub>=0.18.0 from https://files.pythonhosted.org/packages/05/09/1945ca6ba3ad8ad6e2872ba682ce8d68c5e63c8e55458ed8ab4885709f1d/huggingface_hub-0.19.4-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 32.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
  Attempti

In [ ]:
import pandas as pd
import numpy as np
import json
from joblib import dump, load

import os
import random
from tqdm.notebook  import tqdm

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

from transformers import (AutoConfig,
                          AutoTokenizer,
                          DefaultDataCollator,
                          AutoModelForQuestionAnswering,
                          TrainingArguments,
                          Trainer,
                          default_data_collator,
                          get_scheduler)

from datasets import load_dataset
import evaluate
import collections
import optuna

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = True

seed_everything(42)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

In [ ]:
dataset = load_dataset(path='/kaggle/input/aiconnect-mrc',data_files='./custom_data.json', split='train')
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['id', 'question', 'context', 'answers'],
    num_rows: 28024
})

In [ ]:
squad = dataset.train_test_split(.2)
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'context', 'answers'],
        num_rows: 22419
    })
    test: Dataset({
        features: ['id', 'question', 'context', 'answers'],
        num_rows: 5605
    })
})

In [ ]:
squad["train"][0]

{'id': 'QUES_UniBPiuRnf',
 'question': '더 나은 삶 지수 중 하나인 공동체 지수에서 어느 나라가 최하위를 기록했니',
 'context': '경제협력개발기구(OECD)가 발표한 ‘더 나은 삶 지수(Better Life Index, BLI)’ 중 하나인 ‘공동체(Community)’ 지수에서 대한민국은 러시아와 브라질을 포함한 36개 국가 중 최하위를 기록했다. 한국 사회에서 공동체가 파괴되어온 주요인으로 다양한 의견이 개진되었으나, 한국도시의 오래된 특징인 아파트와 고립된 생활 공간, 차도와 인도의 구분이 없는 길과 건물 배치가 이웃 간 소통을 가로막고 갈등이 생겨도 이를 원활하게 해결할 수 없는 환경을 조성했다는 지적이 주목을 받았다(김재형, 2016). 도움이 필요할 때 이웃의 관심과 손길을 기대하기 어려운 환경은 고독사, 아동학대, 가정폭력, 자살 등 다양한 사회문제를 방치하고 악화시키는 데 일조한다. 이러한 공동체 와해와 지역현안 문제를 주민들의 참여를 통해 적극적으로 해결하고자 시행된 정책사업 중 하나가 2010년부터 각 지자체에 본격적으로 도입된 마을 만들기 사업이다. 마을 만들기 사업의 유형은 주체와 내용에 따라 다양한 형태로 발현되지만, 그 핵심은 주민참여를 주요 동력으로 공동의 지역문제와 욕구를 해결하고, 공동체 회복을 통해 주민의 삶의 질을 향상시킨다는 정책적 목적으로 귀결된다(박선희, 2014).',
 'answers': {'answer_start': [84], 'text': ['대한민국']}}

In [ ]:
print("Context: ", squad["train"][0]["context"])
print("Question: ", squad["train"][0]["question"])
print("Answer: ", squad["train"][0]["answers"])

Context:  경제협력개발기구(OECD)가 발표한 ‘더 나은 삶 지수(Better Life Index, BLI)’ 중 하나인 ‘공동체(Community)’ 지수에서 대한민국은 러시아와 브라질을 포함한 36개 국가 중 최하위를 기록했다. 한국 사회에서 공동체가 파괴되어온 주요인으로 다양한 의견이 개진되었으나, 한국도시의 오래된 특징인 아파트와 고립된 생활 공간, 차도와 인도의 구분이 없는 길과 건물 배치가 이웃 간 소통을 가로막고 갈등이 생겨도 이를 원활하게 해결할 수 없는 환경을 조성했다는 지적이 주목을 받았다(김재형, 2016). 도움이 필요할 때 이웃의 관심과 손길을 기대하기 어려운 환경은 고독사, 아동학대, 가정폭력, 자살 등 다양한 사회문제를 방치하고 악화시키는 데 일조한다. 이러한 공동체 와해와 지역현안 문제를 주민들의 참여를 통해 적극적으로 해결하고자 시행된 정책사업 중 하나가 2010년부터 각 지자체에 본격적으로 도입된 마을 만들기 사업이다. 마을 만들기 사업의 유형은 주체와 내용에 따라 다양한 형태로 발현되지만, 그 핵심은 주민참여를 주요 동력으로 공동의 지역문제와 욕구를 해결하고, 공동체 회복을 통해 주민의 삶의 질을 향상시킨다는 정책적 목적으로 귀결된다(박선희, 2014).
Question:  더 나은 삶 지수 중 하나인 공동체 지수에서 어느 나라가 최하위를 기록했니
Answer:  {'answer_start': [84], 'text': ['대한민국']}


In [ ]:
train_dataset = squad["train"]
valid_dataset = squad["test"]

In [ ]:
# select samples
train_dataset = train_dataset.select(range(1000))
valid_dataset = valid_dataset.select(range(100))

In [ ]:
MODEL_NAME ='monologg/kobigbird-bert-base'
config = AutoConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/870 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/492k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

In [ ]:
max_length = 1024
stride = 128

def preprocess_training_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride = stride,
        return_offsets_mapping=True,
        return_overflowing_tokens=True,
        padding="max_length",
    )

    #the offset mapping gives us a tuple indicating the sub-token’s start position and end position relative to the original token it was split from
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
      sample_idx = sample_map[i]
      answer = answers[sample_idx]
      start_char = answer["answer_start"][0]
      end_char = answer["answer_start"][0] + len(answer["text"][0])
      sequence_ids = inputs.sequence_ids(i)

      # Find the start and end of the context
      idx = 0
      while sequence_ids[idx] != 1:
          idx += 1
      context_start = idx
      while sequence_ids[idx] == 1:
          idx += 1
      context_end = idx - 1

      # If the answer is not fully inside the context, label it (0, 0)
      if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
          start_positions.append(0)
          end_positions.append(0)
      else:
          # Otherwise it's the start and end token positions
          idx = context_start
          while idx <= context_end and offset[idx][0] <= start_char:
              idx += 1
          start_positions.append(idx - 1)

          idx = context_end
          while idx >= context_start and offset[idx][1] >= end_char:
              idx -= 1
          end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs

In [ ]:
def preprocess_validation_function(exmaples):
    questions = [q.strip() for q in exmaples["question"]]
    inputs = tokenizer(
        questions,
        exmaples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(exmaples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
train_set = train_dataset.map(preprocess_training_function,
                              batched=True,
                              remove_columns= train_dataset.column_names,
                                    load_from_cache_file=False,)

valid_set = valid_dataset.map(preprocess_validation_function,
                              batched=True,
                              remove_columns= valid_dataset.column_names,
                                    load_from_cache_file=False,)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
train_set

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 1000
})

In [ ]:
valid_set

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 100
})

In [ ]:
valid_modelset =valid_set.remove_columns(["example_id", "offset_mapping"])

In [ ]:
train_set.set_format("torch")
valid_modelset.set_format("torch")

In [ ]:
def get_loader(batch_size):
  trainloader = DataLoader(train_set, shuffle=True, batch_size=batch_size, collate_fn=default_data_collator)
  validloader = DataLoader(valid_modelset, shuffle=False, batch_size=batch_size, collate_fn=default_data_collator)
  return trainloader, validloader

In [ ]:
metric = evaluate.load("squad")

In [ ]:
n_best = 20
max_answer_length = 30

def compute_metrics(start_logits, end_logits,features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    if (end_index < start_index
                        or end_index - start_index +1 > max_answer_length):
                      continue
                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],}
                    answers.append(answer)

        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=references)

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

model.safetensors:   0%|          | 0.00/458M [00:00<?, ?B/s]

Some weights of BigBirdForQuestionAnswering were not initialized from the model checkpoint at monologg/kobigbird-bert-base and are newly initialized: ['qa_classifier.intermediate.dense.weight', 'qa_classifier.output.LayerNorm.bias', 'qa_classifier.output.dense.weight', 'qa_classifier.output.dense.bias', 'qa_classifier.intermediate.dense.bias', 'qa_classifier.output.LayerNorm.weight', 'qa_classifier.qa_outputs.bias', 'qa_classifier.qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
accumulation_steps = 8

def train(trainloader, model, optimizer, lr_scheduler, device):
    model.train()
    running_loss = 0.0
    for idx, batch in enumerate(trainloader):
      batch ={k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      loss = outputs.loss
      running_loss += loss.item()
      if (idx +1) % accumulation_steps == 0:
          loss.backward()
          optimizer.step()
          optimizer.zero_grad()
          lr_scheduler.step()
    train_loss = running_loss / len(trainloader)
    return train_loss

In [ ]:
def validate(validloader, model, device):
    model.eval()
    start_logits =[]
    end_logits = []
    with torch.no_grad():
        for idx, batch in enumerate(validloader):
            batch ={k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)

            start_logits.append(outputs.start_logits.detach().cpu().numpy())
            end_logits.append(outputs.end_logits.detach().cpu().numpy())

        start_logits = np.concatenate(start_logits, axis=0)
        end_logits = np.concatenate(end_logits, axis=0)
        results = compute_metrics(start_logits, end_logits, valid_set, valid_dataset)
    return  results["exact_match"], results["f1"]

In [ ]:
# Learning_rate=2e-5
# Weight_decay =  0.0
# num_epochs = 2
# Batch_size = 1
# accumulation_steps = 32
# trainloader, validloader = get_loader(Batch_size)
# num_update_steps_per_epoch = len(trainloader)
# num_training_steps = num_epochs * num_update_steps_per_epoch

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)
# model = model.to(device)

# optimizer = AdamW(model.parameters(), lr=Learning_rate, weight_decay=Weight_decay)
# lr_scheduler = get_scheduler(
#   'linear',
#   optimizer=optimizer,
#   num_warmup_steps=0,
#   num_training_steps=num_training_steps,
#   )
# for epoch in tqdm(range(1, num_epochs+1)):
#     train_loss = train(trainloader, model, optimizer, lr_scheduler, device)
#     em, f1 = validate(validloader, model, device)
#     print("Epoch:{:1d}/{:1d}, Train Loss:{:.4f}, EM:{:.2f}, F1:{:.2f}".format(epoch, num_epochs, train_loss, em, f1 ))

In [ ]:
torch.cuda.empty_cache()

def train_mrc(trial):
    Learning_rate=trial.suggest_float('learning_rate', 5e-7, 5e-3, log=True)
    Weight_decay = trial.suggest_float('weight_decay', 0.0, 0.3)
    num_epochs = 10
    Batch_size = 1
    accumulation_steps = 32
    trainloader, validloader = get_loader(Batch_size)
    num_update_steps_per_epoch = len(trainloader)
    num_training_steps = num_epochs * num_update_steps_per_epoch

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)
    model = model.to(device)

    optimizer = AdamW(model.parameters(), lr=Learning_rate, weight_decay=Weight_decay)
    lr_scheduler = get_scheduler(
      trial.suggest_categorical("SchedulerType", ['linear', 'cosine', 'constant']),
      optimizer=optimizer,
      num_warmup_steps=0,
      num_training_steps=num_training_steps,
      )
    for epoch in tqdm(range(1, num_epochs+1)):
        train_loss = train(trainloader, model, optimizer, lr_scheduler, device)
        em, f1 = validate(validloader, model, device)
        print("Epoch:{:1d}/{:1d}, Train Loss:{:.4f}, EM:{:.2f}, F1:{:.2f}".format(epoch, num_epochs, train_loss, em, f1 ))

    return em


In [ ]:
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction="maximize")
study.optimize(train_mrc, n_trials=20)
dump(study, '/kaggle/working/aiconnect_mrc_optuna.pkl')

[I 2023-12-11 13:16:52,096] A new study created in memory with name: no-name-e99687d9-17fd-4911-b3ea-83266617eff3
Some weights of BigBirdForQuestionAnswering were not initialized from the model checkpoint at monologg/kobigbird-bert-base and are newly initialized: ['qa_classifier.intermediate.dense.weight', 'qa_classifier.output.LayerNorm.bias', 'qa_classifier.output.dense.weight', 'qa_classifier.output.dense.bias', 'qa_classifier.intermediate.dense.bias', 'qa_classifier.output.LayerNorm.weight', 'qa_classifier.qa_outputs.bias', 'qa_classifier.qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch:1/10, Train Loss:5.6704, EM:0.00, F1:2.40


  0%|          | 0/100 [00:00<?, ?it/s]

...

Epoch:9/10, Train Loss:0.5053, EM:72.00, F1:78.12


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2023-12-11 22:24:48,818] Trial 19 finished with value: 70.0 and parameters: {'learning_rate': 3.313973514679921e-05, 'weight_decay': 0.26113053258744606, 'SchedulerType': 'cosine'}. Best is trial 16 with value: 76.0.


Epoch:10/10, Train Loss:0.4410, EM:70.00, F1:77.80


['/kaggle/working/aiconnect_mrc_optuna.pkl']

In [ ]:
study = load('/kaggle/working/aiconnect_mrc_optuna.pkl')
df = study.trials_dataframe()
df.head()

,number,value,datetime_start,datetime_complete,duration,params_SchedulerType,params_learning_rate,params_weight_decay,state
0,0,0.0,2023-12-11 13:16:52.097515,2023-12-11 13:44:52.632563,0 days 00:28:00.535048,constant,8.741268e-04,0.179540,COMPLETE
1,1,0.0,2023-12-11 13:44:52.634285,2023-12-11 14:13:00.929961,0 days 00:28:08.295676,linear,3.805722e-03,0.080957,COMPLETE
2,2,0.0,2023-12-11 14:13:00.931556,2023-12-11 14:40:32.777106,0 days 00:27:31.845550,constant,1.158814e-04,0.055481,COMPLETE
3,3,16.0,2023-12-11 14:40:32.778571,2023-12-11 15:08:16.759862,0 days 00:27:43.981291,linear,7.888511e-07,0.188205,COMPLETE
4,4,64.0,2023-12-11 15:08:16.761403,2023-12-11 15:36:15.377512,0 days 00:27:58.616109,cosine,6.841812e-05,0.278004,COMPLETE


In [ ]:
df

,number,value,datetime_start,datetime_complete,duration,params_SchedulerType,params_learning_rate,params_weight_decay,state
0,0,0.0,2023-12-11 13:16:52.097515,2023-12-11 13:44:52.632563,0 days 00:28:00.535048,constant,8.741268e-04,0.179540,COMPLETE
1,1,0.0,2023-12-11 13:44:52.634285,2023-12-11 14:13:00.929961,0 days 00:28:08.295676,linear,3.805722e-03,0.080957,COMPLETE
2,2,0.0,2023-12-11 14:13:00.931556,2023-12-11 14:40:32.777106,0 days 00:27:31.845550,constant,1.158814e-04,0.055481,COMPLETE
3,3,16.0,2023-12-11 14:40:32.778571,2023-12-11 15:08:16.759862,0 days 00:27:43.981291,linear,7.888511e-07,0.188205,COMPLETE
4,4,64.0,2023-12-11 15:08:16.761403,2023-12-11 15:36:15.377512,0 days 00:27:58.616109,cosine,6.841812e-05,0.278004,COMPLETE
5,5,0.0,2023-12-11 15:36:15.378959,2023-12-11 16:03:55.379665,0 days 00:27:40.000706,constant,5.630247e-04,0.003032,COMPLETE
6,6,66.0,2023-12-11 16:03:55.381234,2023-12-11 16:31:24.561423,0 days 00:27:29.180189,cosine,1.021714e-05,0.115443,COMPLETE
7,7,14.0,2023-12-11 16:31:24.562840,2023-12-11 16:58:58.328127,0 days 00:27:33.765287,linear,8.554897e-07,0.280526,COMPLETE
8,8,73.0,2023-12-11 16:58:58.329397,2023-12-11 17:26:29.081642,0 days 00:27:30.752245,linear,2.013713e-05,0.176219,COMPLETE
9,9,0.0,2023-12-11 17:26:29.083122,2023-12-11 17:54:11.879566,0 days 00:27:42.796444,linear,2.141484e-04,0.104397,COMPLETE


In [ ]:
df.sort_values(by='value',  ascending=False)

,number,value,datetime_start,datetime_complete,duration,params_SchedulerType,params_learning_rate,params_weight_decay,state
16,16,76.0,2023-12-11 20:36:58.436972,2023-12-11 21:03:52.808259,0 days 00:26:54.371287,linear,4.417114e-05,0.297619,COMPLETE
10,10,75.0,2023-12-11 17:54:11.881209,2023-12-11 18:22:01.465938,0 days 00:27:49.584729,linear,1.256597e-05,0.210538,COMPLETE
12,12,74.0,2023-12-11 18:49:16.718203,2023-12-11 19:16:12.200239,0 days 00:26:55.482036,linear,1.528351e-05,0.226465,COMPLETE
8,8,73.0,2023-12-11 16:58:58.329397,2023-12-11 17:26:29.081642,0 days 00:27:30.752245,linear,2.013713e-05,0.176219,COMPLETE
14,14,73.0,2023-12-11 19:43:06.637566,2023-12-11 20:10:01.756350,0 days 00:26:55.118784,linear,3.343270e-05,0.233944,COMPLETE
17,17,70.0,2023-12-11 21:03:52.809888,2023-12-11 21:30:51.232076,0 days 00:26:58.422188,linear,4.582490e-05,0.293820,COMPLETE
15,15,70.0,2023-12-11 20:10:01.757552,2023-12-11 20:36:58.435502,0 days 00:26:56.677950,cosine,3.516977e-06,0.143763,COMPLETE
19,19,70.0,2023-12-11 21:57:49.657459,2023-12-11 22:24:48.818079,0 days 00:26:59.160620,cosine,3.313974e-05,0.261131,COMPLETE
11,11,67.0,2023-12-11 18:22:01.467354,2023-12-11 18:49:16.716751,0 days 00:27:15.249397,linear,1.106637e-05,0.215798,COMPLETE
6,6,66.0,2023-12-11 16:03:55.381234,2023-12-11 16:31:24.561423,0 days 00:27:29.180189,cosine,1.021714e-05,0.115443,COMPLETE
